In [1]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
names =['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df = pd.read_csv('train.csv',header=None,sep=",",names=names, encoding = "ISO-8859-1")

c:\users\susanna xu\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df['toxic_score'] = df[['toxic','severe_toxic','obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)
#drop unnecessary columns
df = df.drop(['id', 'toxic','severe_toxic','obscene', 'threat', 'insult', 'identity_hate'], axis=1)
#drop first row
df.drop(df.index[:1], inplace=True)
#df.drop(df.index[:150000], inplace=True) #smaller data set

In [4]:
print(df)

                                             comment_text toxic_score
150000  , manned with personnel from the respective co...           0
150001  "\nThe addition was a ""wikipedia situation wh...           0
150002  I have the reference listed at the head of the...           0
150003                    ?\n What is a Juliancolton'''''           0
150004  If you were, in fact, planning to appeal your ...           0
150005  "\nUse of New Template\nI have created a templ...           0
150006  Deaths in the White House \n\nDoes anyone else...           0
150007  I'd not call The Dark Knight, more like a crim...           0
150008  So this came up already a few months ago? If t...           0
150009  See United States nationality law#Nationals wh...           0
150010  "\nI wasn't trying to be bad, honest.  I was j...           0
150011  If you have a chance to read this you should i...           1
150012  "\nRyan, these people are not worth your time....           0
150013  Image\n\nThe

In [5]:
df.head(5)

,comment_text,toxic_score
150000,", manned with personnel from the respective co...",0
150001,"""\nThe addition was a """"wikipedia situation wh...",0
150002,I have the reference listed at the head of the...,0
150003,?\n What is a Juliancolton''''',0
150004,"If you were, in fact, planning to appeal your ...",0


In [6]:
bad_words = []
negative_file = open("negative-words.txt", "r")
for line in negative_file:
    bad_words.append(line.strip("\n"))
negative_file.close()
profanity_file = open("profanity-words.txt", "r")
for line in profanity_file:
    if (line.strip("\n") not in bad_words):
        bad_words.append(line.strip("\n"))
profanity_file.close()
print(bad_words)

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted', 'aborts', 'abrade', 'abrasive', 'abrupt', 'abruptly', 'abscond', 'absence', 'absent-minded', 'absentee', 'absurd', 'absurdity', 'absurdly', 'absurdness', 'abuse', 'abused', 'abuses', 'abusive', 'abysmal', 'abysmally', 'abyss', 'accidental', 'accost', 'accursed', 'accusation', 'accusations', 'accuse', 'accuses', 'accusing', 'accusingly', 'acerbate', 'acerbic', 'acerbically', 'ache', 'ached', 'aches', 'achey', 'aching', 'acrid', 'acridly', 'acridness', 'acrimonious', 'acrimoniously', 'acrimony', 'adamant', 'adamantly', 'addict', 'addicted', 'addicting', 'addicts', 'admonish', 'admonisher', 'admonishingly', 'admonishment', 'admonition', 'adulterate', 'adulterated', 'adulteration', 'adulterier', 'adversarial', 'adversary', 'adverse', 'adversity', 'afflict', 'affliction', 'afflictive', 'affront', 'afraid', 'aggravate', 'aggravating', 'aggravation', 'aggression', 'aggress

In [7]:
comment_list = df['comment_text'].to_numpy()
cv = sklearn.feature_extraction.text.CountVectorizer(vocabulary=bad_words)
print(cv.fit_transform(comment_list).toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
